In [19]:
from datasets import load_dataset
import Preprocessor as p
import numpy as np
import nltk
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
from transformers import pipeline
from datasets import load_dataset
from tqdm import tqdm
import re
import torch.nn.functional as  F
import csv
from transformers import AutoModelForCausalLM, AutoTokenizer

from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from nltk.tokenize import word_tokenize
from next_word_prediction import GPT2

In [2]:
# Load the copenlu/answerable_tydiqa dataset
dataset = load_dataset("copenlu/answerable_tydiqa")

train_df = dataset['train'].to_pandas()
train_df = train_df[train_df['language'].isin(['indonesian', 'arabic', 'bengali'])]
train_df = p.DataFramePreprocessor(train_df).df

val_df = dataset['validation'].to_pandas()
val_df = val_df[val_df['language'].isin(['indonesian', 'arabic', 'bengali'])]
val_df = p.DataFramePreprocessor(val_df).df


In [3]:
data = train_df
data['document_length'] = data['document_plaintext'].str.len()
data['is_answerable'] = data['annotations'].apply(lambda x: x.get('answer_start', [-1])[0] != -1)

val_df['is_answerable'] = data['annotations'].apply(lambda x: x.get('answer_start', [-1])[0] != -1)


In [4]:
data = data.drop_duplicates(subset = ['question_text'])
data = data.drop_duplicates(subset = ['document_plaintext'])


In [5]:
bengali = data[data['language'] == 'bengali']
arabic = data[data['language'] == 'arabic']
indonesian = data[data['language'] == 'indonesian']


bengali_val = val_df[val_df['language'] == 'bengali']
arabic_val = val_df[val_df['language'] == 'arabic']
indonesian_val = val_df[val_df['language'] == 'indonesian']


In [6]:
len(indonesian_val)

1191

In [7]:
"""import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import Dataset

# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Sample pandas DataFrame with a column named "text"
# Replace this with your actual pandas DataFrame
data = arabic

# Create a custom dataset using the datasets library
custom_dataset = Dataset.from_pandas(data)

# Tokenize the custom dataset
def tokenize_function(examples):
    return tokenizer(examples["question_text"])"
tokenized_datasets = custom_dataset.map(tokenize_function, batched=True)
"""

'import pandas as pd\nfrom transformers import GPT2Tokenizer, GPT2LMHeadModel\nfrom datasets import Dataset\n\n# Load the GPT-2 tokenizer and model\ntokenizer = GPT2Tokenizer.from_pretrained("gpt2")\nmodel = GPT2LMHeadModel.from_pretrained("gpt2")\n\n# Sample pandas DataFrame with a column named "text"\n# Replace this with your actual pandas DataFrame\ndata = arabic\n\n# Create a custom dataset using the datasets library\ncustom_dataset = Dataset.from_pandas(data)\n\n# Tokenize the custom dataset\ndef tokenize_function(examples):\n    return tokenizer(examples["question_text"])"\ntokenized_datasets = custom_dataset.map(tokenize_function, batched=True)\n'

In [8]:
"""from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Output directory
    num_train_epochs=3,      # Number of training epochs
    per_device_train_batch_size=4,  # Batch size per device during training
    save_steps=500,          # Save checkpoint every X steps
    logging_steps=100,       # Log metrics every X steps
)

# Instantiate Trainer
trainer = Trainer(
    model=model,                     # The model to be trained
    args=training_args,              # Training arguments
    train_dataset=tokenized_datasets, # Training dataset
)

# Fine-tune the model
trainer.train()
"""

'from transformers import Trainer, TrainingArguments\n\n# Define training arguments\ntraining_args = TrainingArguments(\n    output_dir="./results",  # Output directory\n    num_train_epochs=3,      # Number of training epochs\n    per_device_train_batch_size=4,  # Batch size per device during training\n    save_steps=500,          # Save checkpoint every X steps\n    logging_steps=100,       # Log metrics every X steps\n)\n\n# Instantiate Trainer\ntrainer = Trainer(\n    model=model,                     # The model to be trained\n    args=training_args,              # Training arguments\n    train_dataset=tokenized_datasets, # Training dataset\n)\n\n# Fine-tune the model\ntrainer.train()\n'

In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set the model in evaluation mode
model.eval()

# Define a function to generate top N words from a sequence
def generate_top_words(sequence, n=3):
    input_ids = tokenizer.encode(sequence, return_tensors="pt")

    # Generate text from the model
    with torch.no_grad():
        output = model.generate(input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id)

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract top N words with highest probabilities
    words = generated_text.split()
    word_probabilities = []
    for word in words:
        word_id = tokenizer.encode(word, add_special_tokens=False)[0]
        word_probability = torch.softmax(model(torch.tensor([[word_id]]))[0][:, -1], dim=1).max().item()
        word_probabilities.append((word, word_probability))
    top_words = sorted(word_probabilities, key=lambda x: x[1], reverse=True)[:n]

    return top_words

# Define a prompt for generating a random word
prompt = "Generate a random word: "

# Generate and print top 3 words
top_words = generate_top_words(prompt)
print(f"Top 3 words: {top_words}")


Top 3 words: [('second', 0.07366503775119781), ('word:', 0.060055650770664215), ('word.', 0.060055650770664215)]


In [317]:
from googletrans import Translator


# Function to translate Arabic text to English using Google Translate API
def translate_to_english(arabic_text):
    translator = Translator()
    translated = translator.translate(arabic_text, src='ar', dest='en')  # Translate from Arabic to English
    return translated.text

# Apply translation function to the 'arabic_text' column and create a new 'english_translation' column
arabic_val['english_translation'] = arabic_val['question_text'].apply(translate_to_english)

# Print the DataFrame with English translations



KeyboardInterrupt: 

In [321]:
# Function to translate Arabic text to English using Google Translate API
def translate_to_english(arabic_text):
    translator = Translator()
    translated = translator.translate(arabic_text, src='ar', dest='en')  # Translate from Arabic to English
    return translated.text

# Apply translation function to the 'arabic_text' column and create a new 'english_translation' column
arabic_val['english_translation_doc'] = arabic_val['document_plaintext'].apply(translate_to_english)


In [322]:
def translate_to_english(arabic_text):
    translator = Translator()
    translated = translator.translate(arabic_text, src='bn', dest='en')  # Translate from Arabic to English
    return translated.text

#bengali_val['english_translation'] = bengali_val['question_text'].apply(translate_to_english)
bengali_val['english_translation_doc'] = bengali_val['document_plaintext'].apply(translate_to_english)


In [323]:
def translate_to_english(arabic_text):
    translator = Translator()
    translated = translator.translate(arabic_text, src='id', dest='en')  # Translate from Arabic to English
    return translated.text


#indonesian_val['english_translation'] = indonesian_val['question_text'].apply(translate_to_english)
indonesian_val['english_translation_doc'] = indonesian_val['document_plaintext'].apply(translate_to_english)

In [324]:
indonesian_val.to_csv('indonesian_val.csv', index=True)
bengali_val.to_csv('bengali_val.csv', index=True)
arabic_val.to_csv('arabic_val.csv', index=True)


In [24]:
gpt2 = GPT2()
def predict_next_word(sentence, word, top_n=3):
    # Your prediction logic here, return top_n predictions for the word
    # Example: Replace this line with your prediction logic
    predictions = gpt2.predict_next(word, top_n)
    return predictions

In [11]:
import nltk
# Import CSV files into pandas dataframes
indonesian_val = pd.read_csv('indonesian_val.csv', index_col=0)
bengali_val = pd.read_csv('bengali_val.csv', index_col=0)
arabic_val = pd.read_csv('arabic_val.csv', index_col=0)




# Tokenize the 'english_translation' column into normal tokens
bengali_val['tokenized_english_translation'] = bengali_val['english_translation_doc'].apply(lambda x: nltk.word_tokenize(x))
indonesian_val['tokenized_english_translation'] = indonesian_val['english_translation_doc'].apply(lambda x: nltk.word_tokenize(x))
arabic_val['tokenized_english_translation'] = arabic_val['english_translation_doc'].apply(lambda x: nltk.word_tokenize(x))


In [13]:
import pandas as pd

def generate_predictions_dataframe(tokenized_sentences):
    predictions_list = []
    
    for sentence in tokenized_sentences:
        context = []  # Keep track of previous words as context
        sentence_predictions = []
        
        for word in sentence:
            predictions = predict_next_word(context, word, top_n=3)
            sentence_predictions.append((word, predictions))
            context.append(word)  # Update context with the current word
        
        predictions_list.extend(sentence_predictions)
    
    return predictions_list

# Example usage

"""tokenized_sentences = arabic_val['tokenized_english_translation']
predictions_data = generate_predictions_dataframe(tokenized_sentences)
predictions_dataframe_arabic = pd.DataFrame(predictions_data, columns=['Word', 'Predictions'])
"""

"tokenized_sentences = arabic_val['tokenized_english_translation']\npredictions_data = generate_predictions_dataframe(tokenized_sentences)\npredictions_dataframe_arabic = pd.DataFrame(predictions_data, columns=['Word', 'Predictions'])\n"

In [25]:
tokenized_sentences = indonesian_val['tokenized_english_translation']
predictions_data = generate_predictions_dataframe(tokenized_sentences)
predictions_dataframe_indonesian = pd.DataFrame(predictions_data, columns=['Word', 'Predictions'])


In [36]:
tokenized_sentences = bengali_val['tokenized_english_translation']
predictions_data = generate_predictions_dataframe(tokenized_sentences)
predictions_dataframe_bengali = pd.DataFrame(predictions_data, columns=['Word', 'Predictions'])


In [38]:
bengali_avg = get_acc(predictions_dataframe_bengali)
bengali_avg[:10]

[0.0, 0.5, 0.5, 0.0, 0.0, 1.0, 0.5, 0.5, 0.5, 0.5]

In [ ]:
Indo docs: 

[0.0073,
 0.9984,
 0.4897,
 0.0912,
 0.0167,
 0.0293,
 0.0466,
 0.0262,
 0.0204,
 0.01153]

arabisk doc 

[0.3166,
 0.6421,
 0.6142,
 0.4166,
 0.5613,
 0.6123,
 0.5321,
 0.5234,
 0.3543,
 0.2324]

bengali doc

[0.2234, 0.5882, 0.4166, 0.3231, 0.6123, 0.3315, 0.3413, 0.5164, 0.5435, 0.5319]

In [33]:
indonesian_avg = get_acc(predictions_dataframe_indonesian)


[0.9166666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.4166666666666667,
 0.5,
 0.6666666666666666,
 0.6666666666666666,
 0.5,
 0.3333333333333333,
 0.75]

In [34]:
arabic_avg = get_acc(predictions_dataframe_arabic)


NameError: name 'predictions_dataframe_arabic' is not defined

In [15]:
import pandas as pd

def generate_predictions_dataframe(tokenized_sentences):
    predictions_list = []
    
    for sentence in tokenized_sentences:
        context = []  # Keep track of previous words as context
        sentence_predictions = []
        
        for word in sentence:
            predictions = predict_next_word(context, word, top_n=3)
            sentence_predictions.append((word, predictions))
            context.append(word)  # Update context with the current word
        
        predictions_list.extend(sentence_predictions)
    
    return predictions_list

# Example usage

"""tokenized_sentences = arabic_val['tokenized_english']
predictions_data = generate_predictions_dataframe(tokenized_sentences)

# Convert to DataFrame
predictions_dataframe_arabic = pd.DataFrame(predictions_data, columns=['Word', 'Predictions'])

# Print or return the predictions DataFrame
print(predictions_dataframe)
"""



"tokenized_sentences = arabic_val['tokenized_english']\npredictions_data = generate_predictions_dataframe(tokenized_sentences)\n\n# Convert to DataFrame\npredictions_dataframe_arabic = pd.DataFrame(predictions_data, columns=['Word', 'Predictions'])\n\n# Print or return the predictions DataFrame\nprint(predictions_dataframe)\n"

1        [How, many, times, has, Uruguay, won, the, FIF...
6        [Who, is, the, discoverer, of, quartz, or, qua...
18       [How, does, the, fetus, connect, to, the, uter...
21       [Where, is, Sultan, Abdul, Majeed, Mosque, loc...
24                [What, is, the, capital, of, Georgia, ?]
                               ...                        
13300    [How, many, counties, is, Liberia, divided, in...
13304                 [When, was, toothpaste, invented, ?]
13308               [What, is, existential, philosophy, ?]
13312        [How, high, is, the, High, Dam, in, Egypt, ?]
13324     [What, is, my, first, author, ?, K., Rowling, ?]
Name: tokenized_english, Length: 1902, dtype: object

In [ ]:
predictions_dataframe_arabic
predictions_dataframe_bengali

In [241]:
tokenized_sentences = bengali_val['tokenized_english']
predictions_data = generate_predictions_dataframe(tokenized_sentences)

# Convert to DataFrame
predictions_dataframe_bengali = pd.DataFrame(predictions_data, columns=['Word', 'Predictions'])

In [269]:
tokenized_sentences = indonesian_val['tokenized_english']
predictions_data = generate_predictions_dataframe(tokenized_sentences)

# Convert to DataFrame
predictions_dataframe_indonesian = pd.DataFrame(predictions_data, columns=['Word', 'Predictions'])

,Word,Predictions,result,word
0,Who,"[is, 's, are]",0,discovered
1,discovered,"[the, this, that]",0,the
2,the,"[secret, truth, new]",0,American
3,American,"[Revolution, dream, Dream]",0,continent
4,continent,"[was, in, ,]",1,?
...,...,...,...,...
9747,largest,"[city, company, of]",0,manta
9748,manta,"[ray, rays, -]",0,ray
9749,ray,"[cost, have, make]",0,weigh
9750,weigh,"[?, in, ,]",0,<\EOF>


In [30]:
def check_word_list(row):
    return int(any(word in row['Word'] for word in row['Predictions']))

def check_word_list_nan(row):
    if '?' in row['Word']:
        return np.nan
    return int(any(word in row['Word'] for word in row['Predictions']))


In [31]:
def get_acc(df):
    df = df
    df['Word'] = df['Word'].shift(-1)
    df.at[df.index[-1], 'Word'] = '<\\EOF>'
    # Apply the function to each row and store the result in a new column 'result'
    
    df['result'] = df.apply(lambda row: check_word_list(row), axis=1)
    
    # Apply the function to each row and store the result in a new column 'result'
    df['result'] = df.apply(lambda row: check_word_list_nan(row), axis=1)
    # Split the 'result' column for every NaN value and append the sequence of 0s and 1s to a list
    split_results = []
    current_sequence = []
    for value in df['result']:
        if pd.isna(value):
            if current_sequence:
                split_results.append(current_sequence)
                current_sequence = []
        else:
            current_sequence.append(value)

    # Append the last sequence if it exists
    if current_sequence:
        split_results.append(current_sequence)

     # Your list of lists
    lists = split_results.copy()

    # Get the length of the longest sublist
    max_length = max(len(sublist) for sublist in lists)

    # Pad the shorter sublists with 0.0 values to make them equal in length
    padded_lists = [sublist + [0.0] * (max_length - len(sublist)) for sublist in lists]

    # Calculate the average of every position in the lists
    averages = [sum(col) / len(col) for col in zip(*padded_lists)]

    return averages
   

In [305]:
bengali_avg = get_acc(predictions_dataframe_bengali)

In [314]:
bengali_avg[:10]

[0.9777777777777777,
 0.5466666666666666,
 0.6133333333333333,
 0.2311111111111111,
 0.4888888888888889,
 0.4533333333333333,
 0.29777777777777775,
 0.26222222222222225,
 0.17333333333333334,
 0.13333333333333333]

In [315]:
indonesian_avg = get_acc(predictions_dataframe_indonesian)
indonesian_avg[:10]

[0.010067114093959731,
 0.9991610738255033,
 0.4354026845637584,
 0.07718120805369127,
 0.0436241610738255,
 0.05201342281879195,
 0.053691275167785234,
 0.04697986577181208,
 0.02181208053691275,
 0.006711409395973154]

In [311]:
arabic_avg = get_acc(predictions_dataframe_arabic)

In [313]:
arabic_avg = get_acc(predictions_dataframe_arabic)
arabic_avg[:10]

[0.9994756161510225,
 0.7126376507603566,
 0.47614053487152597,
 0.1740954378605139,
 0.33560566334556896,
 0.281069743051914,
 0.1976927110644992,
 0.14735186156266386,
 0.11955951756685894,
 0.07813319349764027]